In [4]:
from torch import nn
from transformers import RobertaModel, RobertaTokenizer

class RoBERTaRewardModel(nn.Module):
    def __init__():
        super().__init__()
        self.roberta = RobertaModel.from_pretrained("roberta-base")
        self.classifier = torch.nn.Linear(768, 1)


    def forward(self, good_input_ids, bad_input_ids, good_attention_mask, bad_attention_mask):

        chosen, rejected = good_input_ids, bad_input_ids

        chosen_reward = self.classifier(self.roberta(chosen, attention_mask=good_attention_mask))
        rejected_reward = self.classifier(self.roberta(rejected, attention_mask=bad_attention_mask))

        return chosen_reward, rejected_reward


/home/adrian/Projects/RLHF4Hate/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:

class RewardLoss(nn.Module):
    def __init__(self):
        super(CustomLoss, self).__init__()

    def forward(self, chosen_reward, rejected_reward):
        # Calculate the difference and apply sigmoid
        return -torch.log(torch.sigmoid(chosen_reward - rejected_reward))

In [6]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer

class SentencePairDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        """
        Args:
            data (list of tuples): Each tuple contains a context, good example, and bad example string.
            tokenizer (RobertaTokenizer): RoBERTa tokenizer for encoding inputs.
            max_length (int): Maximum length of tokenized inputs.
        """
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Unpack the tuple containing context, good example, and bad example
        context, good_example, bad_example = self.data[idx]

        # Format as required: "[CLS] Context [SEP] Good example [SEP]" and "[CLS] Context [SEP] Bad example [SEP]"
        good_input = f"<s> {context} </s> {good_example} </s>"
        bad_input = f"<s> {context} </s> {bad_example} </s>"

        # Tokenize each input with RoBERTa's tokenizer
        good_encoding = self.tokenizer(
            good_input, padding='max_length', truncation=True, max_length=self.max_length, return_tensors="pt"
        )
        bad_encoding = self.tokenizer(
            bad_input, padding='max_length', truncation=True, max_length=self.max_length, return_tensors="pt"
        )

        # Extract input IDs and attention masks, and squeeze to remove extra dimensions
        good_input_ids = good_encoding['input_ids'].squeeze()
        good_attention_mask = good_encoding['attention_mask'].squeeze()
        
        bad_input_ids = bad_encoding['input_ids'].squeeze()
        bad_attention_mask = bad_encoding['attention_mask'].squeeze()

        return {
            "good_input_ids": good_input_ids,
            "bad_input_ids": bad_input_ids,
            "good_attention_mask": good_attention_mask,
            "bad_attention_mask": bad_attention_mask
        }

# Example data: List of tuples with (context, good example, bad example)
data = [
    ("The sun is shining", "It is a bright day", "It is raining"),
    ("The car is fast", "It accelerates quickly", "It moves slowly"),
    # Add more data points as needed
]

# Initialize the tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Create dataset and dataloader
dataset = SentencePairDataset(data, tokenizer, max_length=64)
dataloader = DataLoader(dataset, batch_size=100, shuffle=True)



In [7]:
class RewardModelTrainer:
    def __init__(model, dataloader, epochs, lr):
        self.optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)
        self.criterion = RewardLoss()
        self.model = model
        self.dataloader = dataset
        self.epochs = epochs

    def train(self):
        self.model.train()

        best_loss = float('inf')  # Initialize best loss as infinity
        checkpoint_path = "best_model_checkpoint.pth"  # Path to save the best model

        for e in range(self.epochs):
            for batch in self.dataloader:
                self.optimizer.zero_grad()

                # Forward pass through the model
                chosen_reward, rejected_reward = self.model(**batch)

                # Calculate the loss (you can use your custom loss function here)
                loss = self.criterion(chosen_reward, rejected_reward)

                # Backward pass
                loss.backward()

                # Optimize the model
                self.optimizer.step()

                print(f"Epoch {e+1}/{self.epochs}, Loss: {loss.item()}")

                # Save the model checkpoint if we have a new best loss
                if loss.item() < best_loss:
                    best_loss = loss.item()
                    print(f"New best loss: {best_loss}. Saving model checkpoint...")
                    torch.save(self.model.state_dict(), checkpoint_path)

        print(f"Training finished. Best model saved to {checkpoint_path}")
